In [1]:
import pandas as pd
import numpy as np

In [3]:
import confuse

In [ ]:
pipeline:
get dataset path -> get from config 
download
extract


In [8]:
def get_config():
    config = confuse.Configuration("config", __name__)
    config.set_file("../config/config.yaml")
    dataset_name = config["dataset"]["dataset_name"].get(str)
    output_dir = config["dataset"]["output_dir"].get(str)
    return {
        "dataset_name": dataset_name,
        "output_dir": output_dir
    }

get_config()

{'dataset_name': 'prashant268/chest-xray-covid19-pneumonia',
 'output_dir': 'dataset'}

In [3]:
import confuse

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D
from tensorflow.keras.layers import Concatenate, Input
from tensorflow.keras.callbacks import ModelCheckpoint
import yaml

config = confuse.Configuration("Covid19Detection", __name__)
config.set_file("../config/config.yaml")

# Extract parameters from the config file
input_shape = tuple(config["model"]["input_shape"].get(list))
parallel_kernels = config["model"]["parallel_kernels"].get(list)
conv2d_filters = config["model"]["conv2d_filters"].get(list)
conv2d_kernel_size = tuple(config["model"]["conv2d_kernel_size"].get(list))
pool_size = tuple(config["model"]["pool_size"].get(list))
dense_units = config["model"]["dense_units"].get(list)
dropout_rate = config["model"]["dropout_rate"].get(float)
output_units = config["model"]["output_units"].get(int)
output_activation = config["model"]["output_activation"].get(str)

loss = config["training"]["loss"].get(str)
optimizer = config["training"]["optimizer"].get(str)
metrics = config["training"]["metrics"].get(list)

# Input layer
inp = Input(shape=input_shape)
convs = []

# Add parallel Conv2D layers
for kernel_size in parallel_kernels:
    conv = Conv2D(conv2d_filters[0], kernel_size, padding='same', activation='relu', strides=1)(inp)
    convs.append(conv)

# Concatenate the outputs of parallel Conv2D layers
out = Concatenate()(convs)
conv_model = Model(inp, out)

# Build the sequential model
model = Sequential()
model.add(conv_model)

model.add(Conv2D(conv2d_filters[1], conv2d_kernel_size, activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Conv2D(conv2d_filters[2], conv2d_kernel_size, activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Flatten())
model.add(Dropout(dropout_rate))
model.add(Dense(dense_units[0], activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(dense_units[1], activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(output_units, activation=output_activation))

# Compile the model
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Print the model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ functional_12 (Functional)      │ (None, 50, 50, 384)    │        11,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 48, 48, 64)     │       221,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 22, 22, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 11, 11, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3872)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 3872)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       495,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 754,850 (2.88 MB)

 Trainable params: 754,850 (2.88 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
import os

# Define the local dataset path
dataset_path = r'..\artifacts\dataset\Data\train'

# Extract directory names and assign labels
categories = []
labels = {}
label_counter = 0

# List directories in the dataset path
for category in os.listdir(dataset_path):
    if category != 'PNEUMONIA':
        category_path = os.path.join(dataset_path, category)
        if os.path.isdir(category_path):  # Ensure it's a directory
            categories.append(category)
            labels[category] = label_counter
            label_counter += 1

# Print directory names and their corresponding labels
for category in categories:
        print(category)

print(labels)

COVID19
NORMAL
{'COVID19': 0, 'NORMAL': 1}


In [25]:
import os
import cv2
import numpy as np

# Define parameters
img_size = 100
data = []
target = []

# Define the local dataset path
dataset_path = r'..\artifacts\dataset\Data\train'

# Iterate through categories (directories)
for category in categories:
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):  # Ensure it's a directory
        print(category)
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            try:
                # Read and preprocess the image
                img = cv2.imread(img_path, cv2.IMREAD_COLOR)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                resized = cv2.resize(gray, (img_size, img_size))
                data.append(resized)
                target.append(category)  # Use category as the label
            except Exception as e:
                print('Exception:', e)

# Convert data and target to numpy arrays
data = np.array(data)
target = np.array(target)

print("Data shape:", data.shape)
print("Target shape:", target.shape)

COVID19
NORMAL
Data shape: (1726, 100, 100)
Target shape: (1726,)


In [26]:
data = np.array(data) / 255.0

# Reshape your data
data = np.reshape(data, (data.shape[0], img_size, img_size, 1))


new_target = [labels[i] for i in target]


In [27]:
np.save('data',data)
np.save('target',new_target)
data=np.load('data.npy')
target=np.load('target.npy')

In [28]:
data.shape[1:]

(100, 100, 1)